In [ ]:

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import info
import os

# Function to extract k-mer features
def get_kmer_features(sequence, k=3):
    kmers = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        kmers[kmer] = kmers.get(kmer, 0) + 1
    return kmers

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-promoter', 'Promoter'], yticklabels=['Non-promoter', 'Promoter'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'confusion_matrix_{model_name.lower().replace(" ", "_")}.png')
    plt.close()

# Function to plot ROC curve
def plot_roc_curve(y_true, y_prob, model_name, ax):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Download and load UCI Genomic Benchmarks dataset
dataset_name = "human_nontata_promoters"
version = 0

# Check dataset info
info(dataset_name, version)

# Download dataset if not already present
download_dataset(dataset_name, version=version, use_cloud_cache=True)

# Load dataset
data_path = os.path.join(os.path.expanduser("~"), ".genomic_benchmarks", dataset_name)
sequences = []
labels = []

# Load train and test sets
for split in ["train", "test"]:
    for class_name in ["negative", "positive"]:
        class_path = os.path.join(data_path, split, class_name)
        label = 1 if class_name == "positive" else 0
        for file_name in os.listdir(class_path):
            if file_name.endswith(".txt"):
                with open(os.path.join(class_path, file_name), "r") as f:
                    seq = f.read().strip()
                    sequences.append(seq)
                    labels.append(label)

# Create k-mer feature matrix
k = 3
all_kmers = set()
for seq in sequences:
    kmers = get_kmer_features(seq, k)
    all_kmers.update(kmers.keys())

# Convert sequences to feature vectors
X = np.zeros((len(sequences), len(all_kmers)))
kmer_list = list(all_kmers)
for i, seq in enumerate(sequences):
    kmers = get_kmer_features(seq, k)
    for kmer, count in kmers.items():
        if kmer in kmer_list:
            X[i, kmer_list.index(kmer)] = count

y = np.array(labels)

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

# Train and evaluate models
results = {}
for model_name, model in models.items():
    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    print(f"{model_name} Accuracy: {accuracy:.2f}")

    # Plot confusion matrix
    plot_confusion_matrix(y_test, y_pred, model_name)

# Plot ROC curves for all models
fig, ax = plt.subplots(figsize=(8, 6))
for model_name, model in models.items():
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)
    plot_roc_curve(y_test, y_prob, model_name, ax)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves')
ax.legend(loc='lower right')
plt.savefig('roc_curves.png')
plt.close()

# Plot model comparison
plt.figure(figsize=(8, 5))
sns.barplot(x=list(results.values()), y=list(results.keys()), palette='Blues_d')
plt.xlabel('Accuracy')
plt.title('Model Performance Comparison')
plt.savefig('model_comparison.png')
plt.close()

# Save feature importances for Random Forest
rf_model = models["Random Forest"]
feature_importance = pd.DataFrame({
    'kmer': kmer_list,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)
feature_importance.to_csv('feature_importance_rf.csv', index=False)